In [1]:
%matplotlib qt
import numpy as np
import matplotlib.pyplot as plt
import hyperspy.api as hs

In [11]:
dpau = hs.load("128x128x12_12cm_1deg_step3_1nm_NBDalpha5_speda.hspy")

ERROR:numba.cuda.cudadrv.driver:Call to cuInit results in UNKNOWN_CUDA_ERROR


In [ ]:
dpau = hs.load('spedbBREG.hspy')

In [12]:
dpau.plot(vmax = 30, cmap = 'viridis')

In [ ]:
dpau.center_direct_beam(method="blur",half_square_width = 30, sigma = 3)

In [ ]:
from skimage import filters

def crop_minimum(image, minimum=3):
    copied = image.copy()
    copied[copied <= minimum] = 0.
    return copied

In [ ]:
procdat = dpau.inav[52,71].subtract_diffraction_background(method="difference of gaussians",
                                                            min_sigma=8,
                                                            max_sigma=10, 
                                                            lazy_result=False)
procdat = procdat.map(filters.gaussian, sigma=1, inplace=False)
procdat = procdat.map(crop_minimum, minimum = 1, inplace=False)
procdat = procdat.map(filters.gaussian, sigma=0.5, inplace=False)

In [ ]:
procdat.plot(vmax = 30)

In [ ]:
dpau = dpau.subtract_diffraction_background(method="difference of gaussians",
                                                            min_sigma=8,
                                                            max_sigma=10, 
                                                            lazy_result=True)

In [ ]:
dpau.map(filters.gaussian, sigma=1)
dpau.map(crop_minimum, minimum = 1)
dpau.map(filters.gaussian, sigma=0.5)

In [ ]:
dpau.compute()

In [ ]:
dpau.plot(vmax = 30)

In [ ]:
dpau.save('spedaBREG.hspy')

In [ ]:
test_patterns = hs.signals.Signal2D((dpau.inav[43,68].data,
                                     dpau.inav[76,40].data,
                                     dpau.inav[95,92].data,
                                     dpau.inav[97,55].data,
                                    ))

In [ ]:
import diffpy
from diffsims.generators.rotation_list_generators import get_beam_directions_grid
from diffsims.generators.diffraction_generator import DiffractionGenerator
from diffsims.libraries.structure_library import StructureLibrary
from diffsims.generators.library_generator import DiffractionLibraryGenerator
from diffsims.libraries.diffraction_library import DiffractionLibrary
from pyxem.utils.calibration_utils import find_diffraction_calibration

In [ ]:
%%capture burn
import diffpy
def template_log_func(x):
    return np.log10(x + 0.01)

resolution = 1 
grid = get_beam_directions_grid("cubic", resolution, mesh="spherified_cube_edge")
structure_matrix = diffpy.structure.loadStructure("Data/Au_mp-81_conventional_standard.cif")

diff_gen = DiffractionGenerator(accelerating_voltage=200,
                                precession_angle=1,
                                scattering_params=None,
                                shape_factor_model="linear",
                                minimum_intensity=0.1,
                                )

lib_gen = DiffractionLibraryGenerator(diff_gen)
library_phases = StructureLibrary(["phase"], [structure_matrix], [grid])

found_cal, corrline, found_cals = find_diffraction_calibration(test_patterns,0.0093,library_phases,lib_gen,20,
                                                               max_excitation_error= 0.08,
                                                               intensity_transform_function=template_log_func,
                                                               delta_r = 0.5,
                                                               max_r = np.sqrt(128**2+128**2) * 0.90
                                                              )

In [ ]:
from diffsims.generators.rotation_list_generators import get_beam_directions_grid
resolution = 0.5 
grid_cub = get_beam_directions_grid("cubic", resolution, mesh="spherified_cube_edge")
print("Number of patterns: ", grid_cub.shape[0])

In [ ]:
import diffpy
from diffsims.libraries.structure_library import StructureLibrary
from diffsims.generators.diffraction_generator import DiffractionGenerator
from diffsims.generators.library_generator import DiffractionLibraryGenerator

In [ ]:
# Parameters necessary for simulating a template library
diffraction_calibration = 0.0093
half_shape = (dpau.data.shape[-2]//2, dpau.data.shape[-1]//2)
reciprocal_radius = np.sqrt(half_shape[0]**2 + half_shape[1]**2)*diffraction_calibration

In [ ]:
structure_matrix = diffpy.structure.loadStructure("Data/Au_mp-81_conventional_standard.cif")
grid = get_beam_directions_grid("cubic", 0.5, mesh="spherified_cube_edge")
diff_gen = DiffractionGenerator(accelerating_voltage=200,
                                precession_angle=1,
                                scattering_params=None,
                                shape_factor_model="linear",
                                minimum_intensity=0.1,
                                )

lib_gen = DiffractionLibraryGenerator(diff_gen)

library_phases = StructureLibrary(["Au"], [structure_matrix], [grid])

diff_lib = lib_gen.get_diffraction_library(library_phases,
                                           calibration=diffraction_calibration,
                                           reciprocal_radius=reciprocal_radius,
                                           half_shape=half_shape,
                                           with_direct_beam=False,
                                           max_excitation_error=0.1)

In [ ]:
from pyxem.utils import indexation_utils as iutls
from pyxem.utils import plotting_utils as putls

In [ ]:
def template_log_func(x):
    return np.log(x + 0.01)

In [ ]:
simulations = diff_lib["Au"]["simulations"]
image = dpau.inav[52,71].data
frac_keep = 1
n_keep = None
n_best = 1
delta_r = 1
delta_theta = 1
max_r = None
intensity_transform_function = template_log_func
find_direct_beam = False
direct_beam_position = None
normalize_image = True
normalize_templates = True
indices_n, angles_n, correlations_n, signs_n = iutls.get_n_best_matches(image,
                                                 simulations,
                                                 n_best,
                                                 frac_keep,
                                                 n_keep,
                                                 delta_r,
                                                 delta_theta,
                                                 max_r,
                                                 intensity_transform_function,
                                                 find_direct_beam,
                                                 direct_beam_position,
                                                 normalize_image,
                                                 normalize_templates,
                                                )

In [ ]:
correlations_n

In [ ]:
mirrored = signs_n[0] == -1
x = putls.plot_template_over_pattern(image,
                                 simulations[indices_n[0]],
                                 in_plane_angle=angles_n[0],
                                 coordinate_system = "cartesian", 
                                 size_factor = 10,
                                 vmax=20, 
                                 max_r = 200,
                                 mirrored_template=mirrored,
                                 find_direct_beam=False,
                                 cmap = "viridis"
                                )

In [ ]:
from pyxem.utils import indexation_utils as iutls
from pyxem.utils import plotting_utils as putls

In [ ]:
def template_log_func(x):
    return np.log10(x + 0.01)

In [ ]:
frac_keep = 1
n_best = 20
n_keep = None

delta_r = 0.5
delta_theta = 0.5
max_r = None
intensity_transform_function = None
direct_beam_position = None
normalize_image = True
normalize_templates = True

result, phasedict = iutls.index_dataset_with_template_rotation(dpau,
                                                    diff_lib,
                                                    phases = ["Au"],
                                                    n_best = n_best,
                                                    frac_keep = frac_keep,
                                                    n_keep = n_keep,
                                                    delta_r = delta_r,
                                                    delta_theta = delta_theta,
                                                    max_r = None,
                                                    intensity_transform_function=intensity_transform_function,
                                                    normalize_images = normalize_image,
                                                    normalize_templates=normalize_templates,
                                                    target='gpu'
                                                    )

In [3]:
from orix import plot
from orix.crystal_map import CrystalMap, Phase, PhaseList
from orix.io import load, save
from orix.quaternion import Orientation, Rotation, symmetry
from orix.vector import Vector3d

In [ ]:
from pyxem.signals.indexation_results import results_dict_to_crystal_map

In [ ]:
xmap = results_dict_to_crystal_map(result,phasedict)

In [4]:
xmap = load("Data/AuNP0xmapBGRED.h5") 

In [5]:
xmap.phases[0].space_group = 225

In [8]:
ckey_m3m = plot.IPFColorKeyTSL(xmap.phases["Au"].point_group, direction=Vector3d.xvector())
rgb_Aux = ckey_m3m.orientation2color(xmap["Au"].orientations)
ckey_m3m = plot.IPFColorKeyTSL(xmap.phases["Au"].point_group, direction=Vector3d.yvector())
rgb_Auy = ckey_m3m.orientation2color(xmap["Au"].orientations)
ckey_m3m = plot.IPFColorKeyTSL(xmap.phases["Au"].point_group, direction=Vector3d.zvector())
rgb_Auz = ckey_m3m.orientation2color(xmap["Au"].orientations)

In [9]:
rgb_all = np.zeros((xmap.size, 3))
rgb_all[xmap.phase_id == 0] = rgb_Cuz
rgb_all[xmap.phase_id == -1] = [0,0,0]

xmap.plot(rgb_all, remove_padding=True)

In [10]:
xmap.scan_unit()

Phase    Orientations  Name  Space group  Point group  Proper point group     Color
    0  16384 (100.0%)    Au        Fm-3m         m-3m                 432  tab:blue
Properties: correlation, mirrored_template, template_index
Scan unit: px

In [ ]:
fig = plt.figure()
ax0 = fig.add_subplot(311, projection="plot_map")
ax1 = fig.add_subplot(312, projection="plot_map")
ax2 = fig.add_subplot(313, projection="plot_map")
ax0.set_title("X")
ax1.set_title("Y")
ax2.set_title("Z")
ax0.plot_map(xmap, rgb_Aux, scalebar = False)
ax1.plot_map(xmap, rgb_Auy, scalebar = False)
ax2.plot_map(xmap, rgb_Auz, scalebar = False)

In [ ]:
ckey_m3m = plot.IPFColorKeyTSL(xmap.phases["Au"].point_group, direction=Vector3d.xvector())
rgb_Aux = ckey_m3m.orientation2color(xmap2["Au"].orientations)
ckey_m3m = plot.IPFColorKeyTSL(xmap.phases["Au"].point_group, direction=Vector3d.yvector())
rgb_Auy = ckey_m3m.orientation2color(xmap2["Au"].orientations)
ckey_m3m = plot.IPFColorKeyTSL(xmap.phases["Au"].point_group, direction=Vector3d.zvector())
rgb_Auz = ckey_m3m.orientation2color(xmap2["Au"].orientations)

In [ ]:
fig = plt.figure()
ax0 = fig.add_subplot(311, projection="plot_map")
ax1 = fig.add_subplot(312, projection="plot_map")
ax2 = fig.add_subplot(313, projection="plot_map")
ax0.set_title("X")
ax1.set_title("Y")
ax2.set_title("Z")
ax0.plot_map(xmap2, rgb_Cux, scalebar = False)
ax1.plot_map(xmap2, rgb_Cuy, scalebar = False)
ax2.plot_map(xmap2, rgb_Cuz, scalebar = False)

In [ ]:
fig = plt.figure()
ax0 = fig.add_subplot(111, projection="plot_map")
ax0.set_title("Z")
ax0.plot_map(xmap, rgb_all, scalebar = False, remove_padding=True)

In [ ]:
xmap["Au"].plot(rgb_Cuz, scalebar = False)

In [ ]:
xmap0.plot(value = xmap0.correlation[:,0])

In [ ]:
xmap.phases.add_not_indexed()

In [ ]:
xmap[(xmap.correlation[:,0] < -0.003)].phase_id = -1

In [ ]:
xmap0.plot()

In [ ]:
rgb_Cuz = ckey_m3m.orientation2color(xmap["Au"][(xmap.correlation[:,0] > 0)].orientations)

In [ ]:
xmap['Au'].plot(value = xmap.correlation[:,0], scalebar = False)

In [ ]:
orifirst = xmap['Au'].orientations
store = np.zeros((xmap['Au'].size,15))
normcorrs = xmap['Au'].correlation[:,:]/np.linalg.norm(xmap['Au'].correlation[:,:]) 
for i in range (15):
    corr_weight = normcorrs[:,i]/normcorrs[:,0]
    # corr_weight = 1 # for unweighted, not recommened
    orinext = Orientation(xmap['Au'].rotations[:,i],symmetry=symmetry.Oh)
    store[:,i] = np.degrees((orinext - orifirst).angle.data) * abs(corr_weight)

mean_misori_weighted = np.mean(store[:,1::], axis = 1)

In [ ]:
xmap['Au'].plot(value = mean_misori_weighted,scalebar = False, vmax = 10, colorbar = True)

In [ ]:
normcorrs = xmap['Au'].correlation[:,:]/np.linalg.norm(xmap['Au'].correlation[:,:]) 
reliability = normcorrs[:,0] - normcorrs[:,1] 

In [ ]:
xmap['Au'].plot(value = reliability,scalebar = False, colorbar = True)

In [ ]:
n= np.linspace(0,4,num = 5)
n

In [ ]:
orifirst = xmap.orientations
store = np.zeros((xmap["Au"].size,20))
for i in range (20):
    corr_weight = xmap.correlation[:,i]/xmap.correlation[:,0]
    orinext = Orientation(xmap.rotations[:,i],symmetry=symmetry.Oh)
    store[:,i] = np.degrees((orinext - orifirst).angle.data) * corr_weight

mean_misori_weighted = np.mean(store[:,1::], axis = 1)

In [ ]:
xmap['Au'].plot(value = mean_misori_weighted,scalebar = False,vmin = 0, vmax = 10, colorbar = True)

In [ ]:
xmap.orientations.scatter(projection= 'ipf', direction = Vector3d(((1, 0, 0),(0,1,0),(0,0,1))),c = n, cmap = 'viridis')

In [ ]:
xmap.rotations[0:2].scatter(projection= 'ipf', direction = Vector3d(((1, 0, 0),(0,1,0),(0,0,1))),c = n, cmap = 'viridis')

In [ ]:
Orientation(xmap.rotations[0:3,0:10], symmetry=symmetry.Oh).scatter(projection= 'ipf', direction = Vector3d(((1, 0, 0),(0,1,0),(0,0,1))),c = n, cmap = 'viridis')

In [ ]:
xmap.orientations.scatter(projection= 'axangle', cmap = 'viridis', c = n)

In [ ]:
save(
    filename="Data/AuNP0xmapBGRED.h5",
    object2write=xmap,
    overwrite=True,  # Default is False
)

In [ ]:
xmap0 = load("Data/AuNP0xmapBGRED.h5") 
xmap1 = load("Data/AuNP1xmapBGRED.h5") 
xmap2 = load("Data/AuNP2xmapBGRED.h5") 

In [ ]:
xmap0.phases.add_not_indexed()
xmap1.phases.add_not_indexed()
xmap2.phases.add_not_indexed()

In [ ]:
xmap0[(xmap0.correlation[:,0] < 0.00005)].phase_id = -1
xmap1[(xmap1.correlation[:,0] < 0.00005)].phase_id = -1
xmap2[(xmap2.correlation[:,0] < 0.0003)].phase_id = -1

In [ ]:
fig = plt.figure()
ax0 = fig.add_subplot(111, projection="plot_map")
ax0.set_title("Z")
ax0.plot_map(xmap0, rgb_all, scalebar = False, remove_padding=True)

In [ ]:
xmap0.plot()

In [ ]:
simulations = diff_lib["Au"]['simulations']
#scan coordinate to check
px = 95
py = 44
# which solution to plot
n_sol = 5

index = np.ravel_multi_index((py,px),xmap.shape)
# query the necessary info from the solution
sim_sol_index = xmap.template_index[index,n_sol]
mirrored_sol = xmap.mirrored_template[index,n_sol]
in_plane_angle = np.rad2deg(xmap.rotations[index,n_sol].to_euler())[0][0]
# # query the appropriate template
sim_sol = simulations[sim_sol_index]



fig = plt.figure()
ax0 = fig.add_subplot(121, projection="plot_map")
ax1 = fig.add_subplot(122)
ax0.set_title("Z")
ax0.plot_map(xmap['Au'],rgb_Cuz, scalebar = False)
ax0.scatter(px,py, marker = 'X', c = 'g', s = 30)
putls.plot_template_over_pattern(dpau.inav[px, py].data,
                                 sim_sol,
                                 ax = ax1,
                                 in_plane_angle=in_plane_angle,
                                 coordinate_system = "cartesian", 
                                 size_factor = 10,
                                 vmax=10,
                                 max_r = 200,
                                 mirrored_template=mirrored_sol,
                                 find_direct_beam=False,
                                 cmap = "inferno_r",
                                 marker_color = "green"
                                )

In [ ]:
xmap['Au'].plot(value = xmap.correlation[:,0],scalebar = False, colorbar = True)

In [ ]:
newbest = np.zeros((result['orientation'].shape[0],result['orientation'].shape[1]),np.int16)

for k in range(20):
    newresult= np.zeros((1,5,3))
    for i in range(result['orientation'].shape[0]):
        for j in range(result['orientation'].shape[1]):
            newresult[i,j] = result['orientation'][i,j,newbest[i,j],:]

    misoris = np.zeros((result['orientation'].shape[0],result['orientation'].shape[1]-1))

    oris = Orientation.from_euler(np.radians(newresult[:,:,:]),symmetry=symmetry.Oh)
    mis = (oris[:,0] - oris[:,1])
    misoris[:,0] = np.degrees((mis).angle.data)
    mis = (oris[:,1] - oris[:,2])
    misoris[:,1] = np.degrees((mis).angle.data)
    mis = (oris[:,0] - oris[:,3])
    misoris[:,2] = np.degrees((mis).angle.data)
    mis = (oris[:,3] - oris[:,4])
    misoris[:,3] = np.degrees((mis).angle.data)
    
    misoris[:,3]


In [ ]:
ckey_m3m = plot.IPFColorKeyTSL(xmap0.phases["Au"].point_group, direction=Vector3d.zvector())
rgb_Cuz0 = ckey_m3m.orientation2color(xmap0["Au"].orientations)

In [ ]:
rgb_all = np.zeros((xmap.size, 3))
rgb_all[xmap0.phase_id == 0] = rgb_Cuz0
rgb_all[xmap0.phase_id == -1] = [0,0,0]

xmap0.plot(rgb_all, remove_padding=True)

In [ ]:
xmap0["Au"].plot(rgb_Cuz0, scalebar = False)

In [ ]:
ckey_m3m = plot.IPFColorKeyTSL(xmap1.phases["Au"].point_group, direction=Vector3d.zvector())
rgb_Cuz1 = ckey_m3m.orientation2color(xmap1["Au"].orientations)

In [ ]:
rgb_all = np.zeros((xmap.size, 3))
rgb_all[xmap1.phase_id == 0] = rgb_Cuz1
rgb_all[xmap1.phase_id == -1] = [0,0,0]

xmap1.plot(rgb_all, remove_padding=True)

In [ ]:
ckey_m3m = plot.IPFColorKeyTSL(xmap2.phases["Au"].point_group, direction=Vector3d.zvector())
rgb_Cuz2 = ckey_m3m.orientation2color(xmap2["Au"].orientations)

rgb_all = np.zeros((xmap.size, 3))
rgb_all[xmap2.phase_id == 0] = rgb_Cuz2
rgb_all[xmap2.phase_id == -1] = [0,0,0]

xmap2.plot(rgb_all, remove_padding=True)

In [ ]:
# particles
pos1 = ((39,74),(31,74),(43,80))
pos2 = ((67,22),(60,31),(72,27))
pos3 = ((72,14),(67,13),(77,20))
pos4 = ((87,22),(80,21),(80,30))
pos5 = ((108,27),(105,28),(112,35))
pos6 = ((122,74),(120,70),(125,83))
pos7 = ((94,94),(84,90),(86,100))
pos8 = ((55,97),(51,95),(59,100))

def simplecluster(pos):
    mis0 = xmap0['Au'].orientations - xmap0[pos[0]].orientations
    mis1 = xmap1['Au'].orientations - xmap1[pos[1]].orientations
    mis2 = xmap2['Au'].orientations - xmap2[pos[2]].orientations

    mean0 = xmap0['Au'][np.rad2deg(mis0.angle.data) < 2].orientations.mean()
    mean1 = xmap1['Au'][np.rad2deg(mis1.angle.data) < 2].orientations.mean()
    mean2 = xmap2['Au'][np.rad2deg(mis2.angle.data) < 2].orientations.mean()

    means = Orientation([mean0.data,mean1.data,mean2.data], symmetry=symmetry.Oh).reshape(3)
    means.map_into_symmetry_reduced_zone()
    # print(np.rad2deg(xmap0[pos[0]].orientations.angle_with(xmap1[pos[1]].orientations).data))
    # print(np.rad2deg(xmap1[pos[1]].orientations.angle_with(xmap2[pos[2]].orientations).data))
    # print(np.rad2deg(xmap0[pos[0]].orientations.angle_with(xmap2[pos[2]].orientations).data))
    print(np.rad2deg(means[0].angle_with(means[1]).data))
    print(np.rad2deg(means[1].angle_with(means[2]).data))
    print(np.rad2deg(means[0].angle_with(means[2]).data))
    return(means)

In [ ]:
means1.map_into_symmetry_reduced_zone().scatter(projection='ipf')

In [ ]:
means1 = means1.map_into_symmetry_reduced_zone()

In [ ]:
print(np.rad2deg((means1[0] - means1[1]).data))
print(np.rad2deg(means1[1].angle_with(means1[2]).data))
print(np.rad2deg(means1[0].angle_with(means1[2]).data))

In [ ]:
means1 = simplecluster(pos1)

In [ ]:
means2 = simplecluster(pos2)
print('-------')
means3 = simplecluster(pos3)
print('-------')
means4 = simplecluster(pos4)
print('-------')
means5 = simplecluster(pos5)
print('-------')
means6 = simplecluster(pos6)
print('-------')
means7 = simplecluster(pos7)
print('-------')
means8 = simplecluster(pos8)
print('-------')

In [ ]:
means1.scatter(projection='ipf')